In [1]:
import numpy as np
import argparse
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from imutils import face_utils
import imutils

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\JitChow\Documents\GitHub\FYP2\Shared\shape_predictor_68_face_landmarks.dat")

In [4]:
def crop_mouth_yawn_dlib(img):
    IMG_SIZE = 50
    coor = (0, 0, 0, 0)
    roi = img.copy()

    try:
        image_array = img.copy()
        gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)
        if(len(rects) < 0):
            return coor, 0
        for (i, rect) in enumerate(rects):
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            coor_i, coor_j = (48, 68)
            coor = (x, y, w, h) = cv2.boundingRect(np.array([shape[coor_i:coor_j]]))
            roi = gray[y:y + h, x:x + w]
            roi = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))

    except Exception as e:
            print(e)
            
    return coor, roi

In [5]:

model = tf.keras.models.load_model(r"C:\Users\JitChow\Documents\GitHub\FYP2\Yawn Detection Codes\models\YawnGray Acc - 0.957 Loss - 0.117.h5")  

In [6]:
import cv2

# define a video capture object
vid = cv2.VideoCapture(0)

while(True):
    ret, frame = vid.read()
    coor, roi = crop_mouth_yawn_dlib(frame)
    
    if coor == (0,0,0,0):
        cv2.imshow('frame', frame)
    else:
        
        
        (x, y, w, h) = (coor[0], coor[1],coor[2],coor[3])
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (36,255,12), 1)

        roi = np.array(roi).reshape(-1,50,50,1)
        roi = roi.astype('float32') / 255.0
        pred = model.predict(roi)

        text = ''
        if pred < 0.5:
            text = 'no yawn'
        else:
            text = 'yawn'

        cv2.putText(frame, text, (x-10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.imshow('frame', frame)
      
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


vid.release()
# Destroy all the windows
cv2.destroyAllWindows()